# Initialize

In [1]:
import os

import numpy as np
import optuna
import torch
from torch.utils.data import DataLoader

from models.RNN_CNN import RNNWithCNN
from utils.dataset import EEGDataset
from utils.training import optuna_train, get_study_callback

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# torch.manual_seed(3407)

In [4]:
%load_ext autoreload
%autoreload 2

## Constants

In [5]:
CP_PATH = "checkpoints/cnn_rnn/"

## Load data

In [6]:
X_train = np.load('data/X_train_valid.npy')
y_train = np.load('data/y_train_valid.npy')
X_test = np.load('data/X_test.npy')
y_test = np.load('data/y_test.npy')

In [7]:
split_idx = 1777
X_train, X_valid = X_train[:split_idx], X_train[split_idx:]
y_train, y_valid = y_train[:split_idx], y_train[split_idx:]

In [8]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_valid shape: {X_valid.shape}")
print(f"y_valid shape: {y_valid.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (1777, 22, 1000)
y_train shape: (1777,)
X_valid shape: (338, 22, 1000)
y_valid shape: (338,)
X_test shape: (443, 22, 1000)
y_test shape: (443,)


In [9]:
train_set = EEGDataset(X_train, y_train, label_smoothing=False)
valid_set = EEGDataset(X_valid, y_valid, label_smoothing=False)
test_set = EEGDataset(X_test, y_test, label_smoothing=False)

In [10]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

# Train

## Setup Optuna

In [11]:
# if os.path.exists(CP_PATH + "CNN_RNN_study.pkl"):
#     print("Loading existing study...")
#     study = joblib.load(CP_PATH + "CNN_RNN_study.pkl")
# else:
#     print("Creating new study...")
study = optuna.create_study(
    study_name="CNN_RNN",
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
    load_if_exists=True,
    storage=os.getenv("RDB_URL"),
)

[I 2024-03-08 02:01:03,418] Using an existing study with name 'CNN_RNN' instead of creating a new one.


In [12]:
def gen_model(trial):
    return RNNWithCNN(trial)


def gen_optimizer(trial, model):
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    return torch.optim.Adam(model.parameters(), weight_decay=weight_decay, lr=lr)


loss_func = torch.nn.CrossEntropyLoss()

## Training

In [13]:
objective = lambda trial: optuna_train(
    trial,
    gen_model, gen_optimizer, loss_func,
    train_loader, valid_loader,
    cp_path=CP_PATH,
    n_epochs=200,
    show_progress_bar=False,
    verbose=False,
)

study.optimize(
    objective, 
    n_trials=200, 
    callbacks=[get_study_callback(CP_PATH)], 
    show_progress_bar=True,
    catch=(ValueError,),
    gc_after_trial=True,
)

  0%|          | 0/200 [00:00<?, ?it/s]

/Users/nickzhou/opt/anaconda3/envs/py3.11/lib/python3.11/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


[I 2024-03-08 02:04:42,275] Trial 40 pruned. 


/Users/nickzhou/opt/anaconda3/envs/py3.11/lib/python3.11/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


[I 2024-03-08 02:06:10,131] Trial 41 finished with value: 0.5798816568047337 and parameters: {'num_cnn_layers': 3, 'cnn_use_bn': True, 'cnn_activation': 'ELU', 'cnn_dropout': 0.33335068650333943, 'cnn_kernel_size': 13, 'cnn_out_channels_0': 86, 'cnn_out_channels_1': 59, 'cnn_out_channels_2': 81, 'use_fc1': False, 'num_rnn_layers': 4, 'rnn_hidden_size': 245, 'rnn_dropout': 0.49929087604342404, 'num_fc_layers_2': 1, 'fc_use_bn_1': True, 'fc_out_features_1_0': 352, 'fc_dropout_1_0': 0.2567213282100878, 'weight_decay': 2.2133030694621662e-05, 'lr': 0.00011049821810148668}. Best is trial 31 with value: 0.6775147928994083.
[I 2024-03-08 02:16:12,548] Trial 42 finished with value: 0.6834319526627219 and parameters: {'num_cnn_layers': 4, 'cnn_use_bn': True, 'cnn_activation': 'ELU', 'cnn_dropout': 0.31692871208310097, 'cnn_kernel_size': 13, 'cnn_out_channels_0': 82, 'cnn_out_channels_1': 125, 'cnn_out_channels_2': 84, 'cnn_out_channels_3': 171, 'use_fc1': False, 'num_rnn_layers': 4, 'rnn_hidden

...

[I 2024-03-08 07:38:20,837] Trial 235 finished with value: 0.6301775147928994 and parameters: {'num_cnn_layers': 5, 'cnn_use_bn': True, 'cnn_activation': 'ELU', 'cnn_dropout': 0.2257172144072064, 'cnn_kernel_size': 13, 'cnn_out_channels_0': 186, 'cnn_out_channels_1': 127, 'cnn_out_channels_2': 173, 'cnn_out_channels_3': 175, 'cnn_out_channels_4': 190, 'use_fc1': False, 'num_rnn_layers': 1, 'rnn_hidden_size': 174, 'rnn_dropout': 0.49884306447962073, 'num_fc_layers_2': 1, 'fc_use_bn_1': True, 'fc_out_features_1_0': 997, 'fc_dropout_1_0': 0.32491949234693684, 'weight_decay': 5.162976598836241e-05, 'lr': 9.750385410922072e-05}. Best is trial 63 with value: 0.7071005917159763.
[I 2024-03-08 07:39:57,997] Trial 236 pruned. 


/Users/nickzhou/opt/anaconda3/envs/py3.11/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4718745427486299 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


[I 2024-03-08 07:43:30,028] Trial 237 finished with value: 0.6745562130177515 and parameters: {'num_cnn_layers': 5, 'cnn_use_bn': True, 'cnn_activation': 'ELU', 'cnn_dropout': 0.18580358943471756, 'cnn_kernel_size': 12, 'cnn_out_channels_0': 180, 'cnn_out_channels_1': 111, 'cnn_out_channels_2': 152, 'cnn_out_channels_3': 171, 'cnn_out_channels_4': 187, 'use_fc1': False, 'num_rnn_layers': 1, 'rnn_hidden_size': 122, 'rnn_dropout': 0.4718745427486299, 'num_fc_layers_2': 1, 'fc_use_bn_1': True, 'fc_out_features_1_0': 989, 'fc_dropout_1_0': 0.29334870171690486, 'weight_decay': 0.00023069227643066827, 'lr': 8.208840353385206e-05}. Best is trial 63 with value: 0.7071005917159763.
[I 2024-03-08 07:44:20,773] Trial 238 pruned. 


/Users/nickzhou/opt/anaconda3/envs/py3.11/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.46981039606335645 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


[I 2024-03-08 07:45:49,717] Trial 239 pruned. 
